In [1]:
#Colab setting
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/RecommanderSystems')

import os
os.chdir('/content/drive/My Drive/RecommanderSystems')

Mounted at /content/drive


In [21]:
from types import SimpleNamespace

config = SimpleNamespace(
    seed = 1,
    data = "Steam.txt", # Beauty.txt / ml-1m.txt
    dropout = 0.5, # 0.2 for ml-1m
    n = 50, # 300 for ml-1m, 50 for others
    d = 40,
    batch_size = 128,
    num_heads = 1, # default
    test_batch_size = 100,
    weight_decay = 5e-4,
    num_blocks = 2,
    lr = 1e-3,
    epoch = 200,
    patience = 20

)

## Dataset

SASRec의 경우 학습을 시킬 때 user 당 item의 시퀀스가 필요

In [27]:
from collections import defaultdict

user_item_sequence_dict = defaultdict(list)

u_max = -1
i_max = -1

data = open(config.data, 'r')
for line in data:
  u, i = line.strip().split(' ') # (user, item) 조합의 txt 파일 읽기
  u = int(u)
  i = int(i)
  user_item_sequence_dict[u].append(i)
  if u > u_max:
    u_max = u
  if i > i_max:
    i_max = i

user_train = {}
user_valid = {}
user_test = {}

for u in user_item_sequence_dict:
  seq = user_item_sequence_dict[u]
  l = len(seq)
  user_valid[u] = []
  user_test[u] = []
  if l < 3:
    padded_seq = [0] * (config.n - l) + seq
    user_train[u] = padded_seq

  elif l >= 52:
    user_train[u] = seq[-config.n - 2:-2]
    user_valid[u] = seq[-2]
    user_test[u] = seq[-1]

  else:
    user_valid[u] = seq[-2]
    user_test[u] = seq[-1]
    padded_seq = [0] * (config.n - l + 2) + seq[:-2]
    user_train[u] = padded_seq

print("u_max", u_max)
print("i_max", i_max)
print(len(user_train[1]))

u_max 334730
i_max 13047
50


# Head

In [57]:
import torch
import torch.nn as nn
import math

class MultiheadAttention(nn.Module):
  def __init__(self, embed_dim, num_heads, kdim = None, vdim = None):
    """

    embed_dim: Dimensionality of the input and output embeddings.
    num_heads: Number of attention heads.

    """
    super().__init__()

    self.embed_dim = embed_dim
    self.num_heads = num_heads

    self.kdim = kdim if kdim is not None else embed_dim
    self.vdim = vdim if vdim is not None else embed_dim

    # 그냥 Q, K, V 사이즈 통일
    self.q_proj = nn.Linear(self.embed_dim, self.embed_dim)
    self.k_proj = nn.Linear(self.kdim, self.embed_dim)
    self.v_proj = nn.Linear(self.vdim, self.embed_dim)
    self.out_proj = nn.Linear(self.embed_dim, self.embed_dim)

    # nn.init.uniform_(self.q_proj.bias)
    # nn.init.uniform_(self.k_proj.bias)
    nn.init.zeros_(self.q_proj.bias)
    nn.init.zeros_(self.k_proj.bias)
    nn.init.zeros_(self.v_proj.bias)
    nn.init.zeros_(self.out_proj.bias)

  def split_heads(self, x):
    """
    Head 늘리는 방식 중에 embedding 사이즈 줄여서 head수 늘리는 방식

    x : (batch_size, -1, embed_dim).

    Returns:
      (batch_size, num_heads, -1, embed_dim // num_heads).

    """
    n_batch = x.shape[0]
    splited = x.reshape(n_batch, -1, self.num_heads, self.embed_dim // self.num_heads)

    return splited.transpose(1, 2) # (B, num_heads, -1, embed_dim // num_heads)

  def scaled_dot_product_attention(self, wq, wk, wv, pad_mask = None):
    """

    wq, wk, wv: (B, num_heads, n_seq, embed_dim // num_heads).

    Returns:
      Scaled Dot-Product Attention  (B, n_seq, embed_dim).
      Average attention weights across heads (B, n_seq, n_key) -시각화에 사용

    """

    n_batch = wq.shape[0]
    d_k = self.embed_dim // self.num_heads

    """
    wq @ wk.T: (B, num_heads, n_seq, n_key)
    pad_mask: (B, n_key)
    softmax @ v: (B, num_heads, n_seq, embed_dim // num_heads)
    """

    wk_t = wk.transpose(2, 3) # (B, num_heads, embed_dim // num_heads, n_key)

    similarity = torch.matmul(wq, wk_t) # (B, num_heads, n_seq, n_key)
    similarity /= math.sqrt(d_k)
    epsilon = -1e9

    # pad_mask가 여기서는 tril로 들어와서 (n, n)으로 들어옴 => (1, 1, n, n)으로 unsqueeze
    if pad_mask is not None:
      similarity = similarity.masked_fill(pad_mask.unsqueeze(0).unsqueeze(1), epsilon) # (B, 1, 1, n_key)

    softmax_sim = torch.softmax(similarity, dim = 3) # key마다 얼마나 비슷 -> softmax (B, num_heads, n_seq, n_key)
    average_att_w = torch.mean(softmax_sim, dim = 1) # head 마다 (B, n_seq, n_key) 평균

    # average_att_w 찍어보면 각 key 마다의 활성화정도 시각화 가능

    attention_value = torch.matmul(softmax_sim, wv) # (B, num_heads, n_seq, embed_dim // num_heads)
    attention_value = attention_value.transpose(1, 2) # (B, n_key, num_heads, embed_dim // num_heads)
    # 사이즈 원상 복구 근데 이게 맞나??? 좀 이상함,,
    attention_value = attention_value.reshape(n_batch, -1, self.embed_dim) # (B, n_key, embed_dim)

    return attention_value, average_att_w

  def forward(self, q, k, v, pad_mask = None):
    """
    q: (B, n_seq, embed_dim)
    k: (B, n_key, kdim)
    v: (B, n_key, vdim)
    pad_mask: (B, n_key)

    Returns:
      output (B, -1, embed_dim)
      average_att_w: (B, n_seq, n_key)

    """
    # print("Multihead input q", q.shape)
    wq = self.q_proj(q) #(B, n_seq, embed_dim)
    wk = self.k_proj(k) #(B, n_key, kdim)
    wv = self.v_proj(v) # #(B, n_key, embed_dim)

    wk = self.split_heads(wk)
    wq = self.split_heads(wq)
    wv = self.split_heads(wv)

    x, attn = self.scaled_dot_product_attention(wq, wk, wv, pad_mask)
    x = self.out_proj(x) # (B, n_key, embed_dim)

    return x, attn

class FeedForward(nn.Module):
  '''
  RELU(SW(1)+b(1))W(2) + b(2)
  W(1) & W(2) : (d, d)

  '''
  def __init__(self, embedding_dim, dropout_rate = 0.5):
    super().__init__()
    self.linear1 = nn.Linear(embedding_dim, embedding_dim) #(d, d)
    self.linear2 = nn.Linear(embedding_dim, embedding_dim)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=dropout_rate)

  def forward(self, x):
    x_ = self.linear1(x)
    x_ = self.dropout(x_) # 이게 중간에 들어가는지 의문,,
    x_ = self.relu(x_)
    x_ = self.linear2(x_)
    x_ = x + self.dropout(x_)

    return x_


In [5]:
pad_mask = torch.tensor([[False, True], [True, False]]) # (2, 2)
print(pad_mask.shape)
similarity = torch.tensor([[[[1, 2]], [[3, 4]]], [[[5, 6]], [[7, 8]]]]) # (2, 2, 1, 2)
print(similarity.shape)
print(similarity)
s =similarity.masked_fill(pad_mask.unsqueeze(1).unsqueeze(2), 1e-9) # (B, 1, 1, n_key)
print(s)

torch.Size([2, 2])
torch.Size([2, 2, 1, 2])
tensor([[[[1, 2]],

         [[3, 4]]],


        [[[5, 6]],

         [[7, 8]]]])
tensor([[[[1, 0]],

         [[3, 0]]],


        [[[0, 6]],

         [[0, 8]]]])


In [ ]:
pad_mask = torch.tensor([False, True]) # (2)
a = torch.tensor([[1, 2, 3], [3, 4, 5]]) # (2, 3)
p = pad_mask.unsqueeze(-1)
print(p)
print(a.masked_fill(p, 0))

tensor([[False],
        [ True]])
tensor([[1, 2, 3],
        [0, 0, 0]])


# Encoder Block

In [58]:
class EncoderBlock(nn.Module):
  def __init__(self, embedding_dim, num_heads, dropout_rate = 0.5):
    super().__init__()

    self.attention = MultiheadAttention(embed_dim = embedding_dim, num_heads = num_heads)
    self.feedforward = FeedForward(embedding_dim)
    self.layernorm = nn.LayerNorm(embedding_dim, eps = 1e-8)
    self.dropout = nn.Dropout(p=dropout_rate)

  def forward(self, x, mask = None):
    '''
    Residual connection
    Q, K, V로 모두 같은 값 사용

    g(x) = x + Dropout(g(LayerNorm(x)))
    '''
    # Self attention Layer

    x_ = self.layernorm(x)
    res1, attn = self.attention.forward(x_, x_, x_, pad_mask = mask)
    res1 = self.dropout(res1)

    x = x + res1

    # FFN
    x_ = self.layernorm(x)
    res2 = self.feedforward(x_)
    res2 = self.dropout(res2)
    x = x + res2

    return x, attn

# DataLoader

In [46]:
from torch.utils.data import Dataset, DataLoader

class UserTrainDataset(Dataset):
  def __init__(self, user_train):
    self.user_train = user_train

    self.users = list(user_train.keys())

  def __len__(self):
    return len(self.users)

  def __getitem__(self, idx):
    user_id = self.users[idx]
    user_data = self.user_train[user_id]

    return user_id, user_data

def collate_fn(batch):
  user_ids, user_data = zip(*batch)
  user_data_tensor = torch.tensor(user_data, dtype=torch.long)  # Assuming user_data is a list of sequences
  return user_ids, user_data_tensor

user_train_dataset = UserTrainDataset(user_train)
user_train_dataloader = DataLoader(user_train_dataset, batch_size= config.batch_size, shuffle=True, collate_fn=collate_fn)


# SASRec model

In [97]:
from re import X
import torch
import torch.nn as nn
import numpy as np

# user_train_dataset = UserTrainDataset(user_train)
# u_max =5
# i_max =10

class SASRec(nn.Module):
  def __init__(self, config, user_train_dataloader = user_train_dataloader, user_train = user_train, user_num = u_max, item_num = i_max):
    super(SASRec, self).__init__()

    self.user_num = user_num
    self.item_num = item_num
    self.config = config
    self.num_blocks = config.num_blocks
    self.embedding_dim = self.config.d
    self.num_heads = config.num_heads
    self.user_seq_dataloader = user_train_dataloader
    self.user_train = user_train

    print("user_num:", user_num) # maybe batch size
    print("item_num:", item_num)
    print("embedding_dim", self.embedding_dim)
    print("num_heads:", self.num_heads)
    print("n:", config.n)

    # M : item embedding (N X d) 인데 item index는 1 부터 시작 => + 1해주기
    self.M = torch.nn.Embedding(self.item_num + 1, self.config.d, padding_idx = 0) # 패딩은 0으로

    self.pos_enc = nn.Parameter(torch.randn(self.config.n, self.config.d)) # learnable PE (n, d)

    self.droptout = nn.Dropout(self.config.dropout)
    self.encoders = nn.ModuleList()

    for block in range(self.num_blocks):
      self.encoders.append(EncoderBlock(self.embedding_dim, self.num_heads))

    self.last_layernorm = torch.nn.LayerNorm(self.embedding_dim, eps=1e-8)

    self.criterion = torch.nn.BCEWithLogitsLoss()
    self.optimizer = torch.optim.Adam(self.parameters(), lr=config.lr)

  def forward(self, user_ids, x):
    '''
    x: (n_batch, config.n)
    user_train[idx]
    '''
    # print("x", x.shape)
    embedded_seq = self.M(torch.LongTensor(x)) # id는 정수들의 sequence (n_batch, config.n, embedding_dim)
    embedded_seq = embedded_seq + self.pos_enc
    pad_mask = torch.BoolTensor(x == 0).unsqueeze(-1)
    # future item 가리기
    attention_mask = ~torch.tril(torch.ones((x.shape[1], x.shape[1]), dtype = torch.bool))

    for encoder in self.encoders:
      embedded_seq = embedded_seq.masked_fill(pad_mask, 0)
      embedded_seq, attn = encoder(embedded_seq, attention_mask)

    output = self.last_layernorm(embedded_seq) # (B, n_key, embed_dim)
    # print("output", output.shape)
    l = 0
    rot_list = []
    rjt_list = []
    seq_l = []
    pos = []
    neg = []

    for l, u in enumerate(user_ids):
      seq = np.zeros([self.config.n], dtype = np.int32)
      pos_seq = np.zeros([self.config.n], dtype = np.int32) # positive sequence : left shifted
      neg_seq = np.zeros([self.config.n], dtype = np.int32) # Random negative
      answer = x[l][-1]
      idx = self.config.n - 1

      for item in reversed(x[l]):
        seq[idx] = item
        pos_seq[idx] = answer
        if answer != 0:
          ns = np.random.randint(1, self.item_num + 1)
          while ns in self.user_train[u]:
            ns = np.random.randint(1, self.item_num + 1)
          neg_seq[idx] = ns
          answer = item
          if idx == 0:
            break
          idx -= 1

      seq_l.append(torch.LongTensor(seq))
      pos.append(torch.LongTensor(pos_seq))
      neg.append(torch.LongTensor(neg_seq))


      # pos_embs = self.M(torch.LongTensor(pos_seq))
      # neg_embs = self.M(torch.LongTensor(neg_seq))

      # print("output", output.shape, "pos_embs", pos_embs.shape)
      # # similarity
      # rot = (output * pos_embs).sum(dim = - 1) # (n_batch, n, d) => (n_batch, n)
      # rjt = (output * neg_embs).sum(dim = - 1) # (n_batch, n, d) => (n_batch, n)

      # indices = np.where(seq != 0)
      #rot_list.append(rot)
      #rjt_list.append(rjt)

    seq_l = torch.stack(seq_l)
    pos_embs = self.M(torch.LongTensor(torch.stack(pos)))
    neg_embs = self.M(torch.LongTensor(torch.stack(neg)))
    rot = (output * pos_embs).sum(dim = - 1)
    rjt = (output * neg_embs).sum(dim = - 1)


    #rot_tensor = torch.stack(rot_list)
    #rjt_tensor = torch.stack(rjt_list)
    indicies = np.where( seq_l != 0)
    #print("rot_tensor", rot.shape)
    return rot, rjt, indicies

  def train(self):
    '''
     batch: (n_batch, seq_length = config.n - 2)
     user_seq -2 / -1 / :-2 valid / test / train
     '''
    for epoch in range(self.config.epoch):
      batch_num = 0
      for idx, batch in self.user_seq_dataloader:
        rot, rjt, indices = self.forward(idx, batch)
        pos_labels, neg_labels = torch.ones(rot.shape), torch.zeros(rjt.shape)
        self.optimizer.zero_grad()
        try:
          loss = self.criterion(rot[indices], pos_labels[indices]) + self.criterion(rjt[indices], neg_labels[indices])
        except:
          print("indice", indices, "pos_labels", pos_labels.shape, neg_labels.shape)
        loss.backward()
        self.optimizer.step()

        if batch_num % 500 == 0:
          print("Epoch", epoch, "Batch_num", batch_num, "loss", loss.item())
        batch_num += 1
      print("loss in epoch {}: {}".format(epoch,loss.item()))
      torch.save(model.state_dict(), 'sasrec_model.pth')
      #if epoch % 20 == 0:


        #print("Evalution in epoch {} iteration {}: {}".format(epoch, loss.item()))




In [98]:
model = SASRec(config)
model.train()

user_num: 334730
item_num: 13047
embedding_dim 40
num_heads: 1
n: 50
Epoch 0 Batch_num 0 loss 4.843395233154297
Epoch 0 Batch_num 50 loss 4.69819974899292
Epoch 0 Batch_num 100 loss 4.323727607727051
Epoch 0 Batch_num 150 loss 3.695662498474121
Epoch 0 Batch_num 200 loss 3.38462495803833
Epoch 0 Batch_num 250 loss 2.6329727172851562
Epoch 0 Batch_num 300 loss 2.4251203536987305
Epoch 0 Batch_num 350 loss 2.179605722427368
Epoch 0 Batch_num 400 loss 2.010875701904297
Epoch 0 Batch_num 450 loss 1.7281379699707031
Epoch 0 Batch_num 500 loss 1.7461698055267334
Epoch 0 Batch_num 550 loss 1.6226662397384644
Epoch 0 Batch_num 600 loss 1.6184031963348389
Epoch 0 Batch_num 650 loss 1.553235650062561
Epoch 0 Batch_num 700 loss 1.500486135482788
Epoch 0 Batch_num 750 loss 1.4583523273468018
Epoch 0 Batch_num 800 loss 1.4331953525543213
Epoch 0 Batch_num 850 loss 1.372255802154541
Epoch 0 Batch_num 900 loss 1.3444125652313232
Epoch 0 Batch_num 950 loss 1.3614170551300049
Epoch 0 Batch_num 1000 los

KeyboardInterrupt: 

In [ ]:
loaded_model = SASRec(config)
loaded_model.load_state_dict(torch.load('sasrec_model.pth'))
loaded_model.train()